# Import findspark 

In [1]:
import findspark
findspark.init()

# Creating a Spark session

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.image import ImageSchema
from pyspark.sql.functions import lit

In [3]:
spark = SparkSession \
.builder \
.appName("Image classification") \
.getOrCreate()

# Loading Images

#Train

In [4]:
image_dir='F:/documents/bigdata/data/min_train/'

In [5]:
Buildings1=ImageSchema.readImages(image_dir + 'buildings/').withColumn('label',lit('B'))
Forest1=ImageSchema.readImages(image_dir + 'forest/').withColumn('label',lit('F'))
Glacier1=ImageSchema.readImages(image_dir + 'glacier/').withColumn('label',lit('G'))
Mountain1=ImageSchema.readImages(image_dir + 'mountain/').withColumn('label',lit('M'))
Sea1=ImageSchema.readImages(image_dir + 'sea/').withColumn('label',lit('C'))
Street1=ImageSchema.readImages(image_dir + 'street/').withColumn('label',lit('S'))

In [ ]:
Buildings1.printSchema()

#Test

In [7]:
image_dir2='F:/documents/bigdata/data/min_test/'

In [8]:
Buildings2=ImageSchema.readImages(image_dir2 + 'buildings/').withColumn('label',lit('B'))
Forest2=ImageSchema.readImages(image_dir2 + 'forest/').withColumn('label',lit('F'))
Glacier2=ImageSchema.readImages(image_dir2 + 'glacier/').withColumn('label',lit('G'))
Mountain2=ImageSchema.readImages(image_dir2 + 'mountain/').withColumn('label',lit('M'))
Sea2=ImageSchema.readImages(image_dir2 + 'sea/').withColumn('label',lit('C'))
Street2=ImageSchema.readImages(image_dir2 + 'street/').withColumn('label',lit('S'))

In [ ]:
Buildings2.show(90)

# Sorting and chracterizing

In [ ]:
Buildings1.select("image.origin", "image.width", "image.height").show(truncate=False)

# Spliting and Merging

In [11]:
train= Buildings1.union(Forest1).union(Glacier1).union(Mountain1).union(Sea1).union(Street1)

In [12]:
display(train)

DataFrame[image: struct<origin:string,height:int,width:int,nChannels:int,mode:int,data:binary>, label: string]

In [13]:
test= Buildings2.union(Forest2).union(Glacier2).union(Mountain2).union(Sea2).union(Street2)

In [14]:
test.show

<bound method DataFrame.show of DataFrame[image: struct<origin:string,height:int,width:int,nChannels:int,mode:int,data:binary>, label: string]>

In [ ]:
#def apply_batch(train):

  # Creates a new TensorFlow graph of computation and imports the model

#  with tf.Graph().as_default() as g:

#    graph_def = tf.GraphDef()

#    graph_def.ParseFromString(model_data_bc.value)

#    tf.import_graph_def(graph_def)

# Runs a tensor flow session that loads the 
#    with tf.Session() as sess:
#        softmax_tensor = sess.graph.get_tensor_by_name('softmax:0')
#        predictions = sess.run(softmax_tensor, {'DecodeJpeg/contents:0':train})
#    return predictions

# MODELING

In [ ]:
from pyspark.ml import Pipeline
from sparkdl import DeepImageFeaturizer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
#import tensorflow.compat.v1 as tf
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# INCEPTION V3

In [ ]:
featurizer = DeepImageFeaturizer(inputCol="image", outputCol="features", modelName="InceptionV3")

In [ ]:
randomforest = RandomForestClassifier(labelCol="label", featuresCol="features")

#Pipelining

In [ ]:
pipeline = Pipeline(stages=[featurizer, randomforest])

In [ ]:
ppipeline = pipeline.fit(train)

In [ ]:
pred = pipeline.transform(test)

In [ ]:
pred.select("file_Path", "labels", "preds").show(100,truncate=False)

In [ ]:
pred_labels = pred.select("preds", "labels")

In [ ]:
Evltr = MulticlassClassificationEvaluator(metricName="accuracy")

In [ ]:
print("accuracy of predictions = " + str(Evltr.evaluate(pred_labels)))